In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
import p2_data as data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

==>Loading data...
==>Data loaded succesfully.


In [11]:
def sigmoid(x):
	'''
		The sigmoid function.
	'''
	return 1. / (1. + np.exp(-x))

def grad_logreg(X, y, W, reg = 0.0):
	'''
		Return the gradient of W for logistic regression.
	'''
	return X.T @ (sigmoid(X @ W) - y) + reg * W

def newton_step(X, y, W, reg = 0.0):
	'''
	Return the change of W according to Newton's method.
	'''
	mu = sigmoid(X @ W)
	g = grad_logreg(X, y, W, reg = reg)
	diag = np.diag(np.squeeze(np.asarray(np.multiply(mu, 1. - mu))))
	H = X.T @ diag @ X + reg * np.eye(X.shape[1])
	d = np.linalg.solve(H, g)
	return d

def NLL(X, y, W, reg = 0.0):
	'''
		Calculate negative log likelihood.
	'''
	mu = sigmoid(X @ W)
	temp = np.multiply(y, np.log(mu)) + np.multiply((1. - y), np.log(1. - mu))
	nll = -sum(temp) + reg / 2 * np.linalg.norm(W) ** 2
	return nll.item(0)

def grad_descent(X, y, reg = 0.0, lr = 1e-4, eps = 1e-6, \
	max_iter = 500, print_freq = 20):
	'''
		X is matrix with dimension m x (n + 1).
		y is label with dimension m x 1.
		reg is the parameter for regularization.
		lr is the learning rate.
		eps is the threshold of the norm for the gradients.
		max_iter is the maximum number of iterations.
		print_freq is the frequency of printing the report.

		Return the optimal weight by gradient descent and
		the corresponding learning objectives.
	'''
	m, n = X.shape
	nll_list = []
	W = np.zeros((n, 1))
	W_grad = np.ones_like(W)
	iter_num = 0
	t_start = time.time()
	while iter_num < max_iter and np.linalg.norm(W_grad) > eps:
		nll = NLL(X, y, W, reg = reg)
		if np.isnan(nll):
			break
		nll_list.append(nll)
		W_grad = grad_logreg(X, y, W, reg = reg)
		W -= lr * W_grad
		if (iter_num + 1) % print_freq == 0:
			print('-- Iteration {} - \
				negative log likelihood {: 4.4f}'.format(iter_num + 1, nll))
		iter_num += 1

	t_end = time.time()

	return W, nll_list

def newton_method(X, y, reg = 0.0, eps = 1e-6, \
	max_iter = 20, print_freq = 5):
	'''
		X is matrix with dimension m x (n + 1).
		y is label with dimension m x 1.
		reg is the parameter for regularization.
		eps is the threshold of the norm for the gradients.
		max_iter is the maximum number of iterations.
		print_freq is the frequency of printing the report.

		Return the optimal weight by Netwon's method and the corresponding
		learning objectives.
	'''
	m, n = X.shape
	nll_list = []
	W = np.zeros((n, 1))
	step = np.ones_like(W)
	print('==> Running Newton\'s method...')
	iter_num = 0
	t_start = time.time()

	while iter_num < max_iter and np.linalg.norm(step) > eps:
		nll = NLL(X, y, W, reg = reg)
		if np.isnan(nll):
			break
		nll_list.append(nll)

		step = newton_step(X, y, W, reg = reg)
		W -= step
		if (iter_num + 1) % print_freq == 0:
			print('-- Iteration {} - \
				negative log likelihood {: 4.4f}'.format(iter_num + 1, nll))
		iter_num += 1

	t_end = time.time()

	return W, nll_list

def predict(X, W):
	'''
		Return the predicted labels.
	'''
	mu = sigmoid(X @ W)
	return (mu >= 0.5).astype(int)

def get_description(X, y, W):
	'''
		X is matrix with dimension m x (n + 1).
		y is label with dimension m x 1.
		W is the weight with dimension (n + 1) x 1.

		Return the accuracy, precision, recall and F-1 score of the prediction.
	'''
	# YOUR CODE GOES BELOW
	m, n = X.shape
	y_pred = predict(X, W)
	count_a, count_p, count_r = 0, 0, 0
	total_p, total_r = 0, 0
	for index in range(m):
		actual, pred = y.item(index), y_pred.item(index)
		if actual == pred:
			count_a += 1
		if actual == 1:
			total_r += 1
			if pred == 1:
				count_r += 1
		if pred == 1:
			total_p += 1
			if actual == 1:
				count_p += 1
	accuracy = 1. * count_a / m
	precision = 1. * count_p / total_p
	recall = 1. * count_r / total_r
	f1 = 2. * precision * recall / (precision + recall)
	return accuracy, precision, recall, f1

def plot_description(X_train, y_train, X_test, y_test):
	'''
		X is matrix with dimension m x (n + 1).
		y is label with dimension m x 1.

		Plot accuracy/precision/recall/F-1 score versus lambda.
		Return the lambda that maximizes accuracy.
	'''
	reg_list = [0., 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 30.0]
	reg_list.sort()
	a_list = []
	p_list = []
	r_list = []
	f1_list = []
	# Run Newton's method or gradient descent
	for index in range(len(reg_list)):
		reg = reg_list[index]
		W_opt, obj = grad_descent(X_train, y_train, reg = reg, \
			lr = 2e-4, print_freq = 100)
		accuracy, precision, recall, f1 = get_description(X_test, y_test, W_opt)
		a_list.append(accuracy)
		p_list.append(precision)
		r_list.append(recall)
		f1_list.append(f1)

	# Generate plots
	a_vs_lambda_plot, = plt.plot(reg_list, a_list)
	plt.setp(a_vs_lambda_plot, color = 'red')
	p_vs_lambda_plot, = plt.plot(reg_list, p_list)
	plt.setp(p_vs_lambda_plot, color = 'green')
	r_vs_lambda_plot, = plt.plot(reg_list, r_list)
	plt.setp(r_vs_lambda_plot, color = 'blue')
	f1_vs_lambda_plot, = plt.plot(reg_list, f1_list)
	plt.setp(f1_vs_lambda_plot, color = 'yellow')
	plt.legend((a_vs_lambda_plot, p_vs_lambda_plot, r_vs_lambda_plot, \
		f1_vs_lambda_plot), ('accuracy', 'precision', 'recall', 'F-1'),\
		 loc = 'best')
	plt.title('Testing descriptions')
	plt.xlabel('regularization parameter')
	plt.ylabel('Metric')
	plt.savefig('hw4pr2a_description.png', format = 'png')
	plt.close()

	# Find the param that maximizes accuracy
	opt_reg_index = np.argmax(a_list)
	reg_opt = reg_list[opt_reg_index]
	return reg_opt

In [12]:
df_train = data.df_train; df_test = data.df_test
X_train = data.X_train; y_train = data.y_train
X_test = data.X_test; y_test = data.y_test

# Problem (a)

### Logistic Regression

In [13]:
# splitting data for logistic regression
df_train_logreg = df_train[df_train.label <= 1]
X_train_logreg = np.array(df_train_logreg[:][[col for \
    col in df_train_logreg.columns if col != 'label']]) / 256.
y_train_logreg = np.array(df_train_logreg[:][['label']])
df_test_logreg = df_test[df_test.label <= 1]
X_test_logreg = np.array(df_test_logreg[:][[col for \
    col in df_test_logreg.columns if col != 'label']]) / 256.
y_test_logreg = np.array(df_test_logreg[:][['label']])

# stacking a column of 1's
X_train_logreg = np.hstack((np.ones_like(y_train_logreg), X_train_logreg))
X_test_logreg = np.hstack((np.ones_like(y_test_logreg), X_test_logreg))

### Gradient Descent w/ Newton Method

In [14]:
W_gd, nll_list_gd = grad_descent(X_train_logreg, y_train_logreg, reg = 1e-6)
W_newton, nll_list_newton = newton_method(X_train_logreg, y_train_logreg, \
    reg = 1e-6)

plt.style.use('ggplot')
nll_gd_plot, = plt.plot(range(len(nll_list_gd)), nll_list_gd)
plt.setp(nll_gd_plot, color = 'red')
nll_newton_plot, = plt.plot(range(len(nll_list_newton)), nll_list_newton)
plt.setp(nll_newton_plot, color = 'green')
plt.legend((nll_gd_plot, nll_newton_plot), ('Gradient descent', 'Newton\'s method'), loc = 'best')
plt.title('Convergence Plot on Binary MNIST Classification')
plt.xlabel('Iteration')
plt.ylabel('NLL')
plt.savefig('hw4pr2a_convergence.png', format = 'png')
plt.close()

-- Iteration 20 - 				negative log likelihood  137.0131
-- Iteration 40 - 				negative log likelihood  108.1078
-- Iteration 60 - 				negative log likelihood  92.1446
-- Iteration 80 - 				negative log likelihood  81.8943
-- Iteration 100 - 				negative log likelihood  74.7108
-- Iteration 120 - 				negative log likelihood  69.3697
-- Iteration 140 - 				negative log likelihood  65.2198
-- Iteration 160 - 				negative log likelihood  61.8818
-- Iteration 180 - 				negative log likelihood  59.1204
-- Iteration 200 - 				negative log likelihood  56.7824
-- Iteration 220 - 				negative log likelihood  54.7646
-- Iteration 240 - 				negative log likelihood  52.9949
-- Iteration 260 - 				negative log likelihood  51.4220
-- Iteration 280 - 				negative log likelihood  50.0080
-- Iteration 300 - 				negative log likelihood  48.7248
-- Iteration 320 - 				negative log likelihood  47.5507
-- Iteration 340 - 				negative log likelihood  46.4689
-- Iteration 360 - 				negative log likelihood  45

/var/folders/pc/1h9xlpvd7rd5xtvc_7rkfznr0000gn/T/ipykernel_88421/2625235106.py:29: RuntimeWarning: divide by zero encountered in log
  temp = np.multiply(y, np.log(mu)) + np.multiply((1. - y), np.log(1. - mu))
/var/folders/pc/1h9xlpvd7rd5xtvc_7rkfznr0000gn/T/ipykernel_88421/2625235106.py:29: RuntimeWarning: invalid value encountered in multiply
  temp = np.multiply(y, np.log(mu)) + np.multiply((1. - y), np.log(1. - mu))


### Statistics Plot and get regularized parameter

In [15]:
reg_opt = plot_description(X_train_logreg, y_train_logreg, X_test_logreg, y_test_logreg)
print('Optimal regularization parameter is {:4.4f}'.format(reg_opt))

-- Iteration 100 - 				negative log likelihood  72.7286
-- Iteration 200 - 				negative log likelihood  48.1433
-- Iteration 300 - 				negative log likelihood  38.4131
-- Iteration 400 - 				negative log likelihood  32.8454
-- Iteration 500 - 				negative log likelihood  28.9308
-- Iteration 100 - 				negative log likelihood  72.8140
-- Iteration 200 - 				negative log likelihood  48.2281
-- Iteration 300 - 				negative log likelihood  38.5023
-- Iteration 400 - 				negative log likelihood  32.9410
-- Iteration 500 - 				negative log likelihood  29.0344
-- Iteration 100 - 				negative log likelihood  73.5791
-- Iteration 200 - 				negative log likelihood  48.9863
-- Iteration 300 - 				negative log likelihood  39.2980
-- Iteration 400 - 				negative log likelihood  33.7935
-- Iteration 500 - 				negative log likelihood  29.9580
-- Iteration 100 - 				negative log likelihood  76.9166
-- Iteration 200 - 				negative log likelihood  52.2491
-- Iteration 300 - 				negative log likelihood  

# Problem (b)

In [24]:
import p2_data as data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import time

In [25]:
def NLL(X, y, W, reg=0.0):
	'''
		Calculate negative log likelihood for softmax regression.
	'''
	# YOUR CODE GOES BELOW
	mu = X @ W # m x k
	exp_mu = np.exp(mu)
	prob = exp_mu / exp_mu.sum(axis=1).reshape(-1, 1)
	groundTruth = y * np.log(prob)
	return -groundTruth.sum(axis=1).sum() + reg * np.diag(W.T @ W).sum()

def grad_softmax(X, y, W, reg=0.0):
	'''
		Return the gradient of W for softmax regression.
	'''
	# YOUR CODE BELOW
	mu = X @ W
	exp_mu = np.exp(mu)
	prob = exp_mu / exp_mu.sum(axis=1).reshape(-1, 1)
	return X.T @ (prob - y) + reg * W

def predict(X, W):
	'''
		Return y_pred with dimension m x 1.
	'''
	# YOUR CODE BELOW
	mu = X @ W
	exp_mu = np.exp(mu)
	prob = exp_mu / exp_mu.sum(axis=1).reshape(-1, 1)
	y_pred = np.argmax(prob, axis=1).reshape(-1, 1)
	return y_pred

def get_accuracy(y_pred, y):
	'''
		Return the percentage of same bits between y_pred and y.
	'''
	diff = (y_pred == y).astype(int)
	accu = 1. * diff.sum() / len(y)
	return accu

def grad_descent(X, y, reg=0.0, lr=1e-5, eps=1e-6, \
	max_iter=500, print_freq=20):
	'''
		X is matrix with dimension m x (n + 1).
		y is label with dimension m x 1.
		reg is the parameter for regularization.
		lr is the learning rate.
		eps is the threshold of the norm for the gradients.
		max_iter is the maximum number of iterations.
		print_freq is the frequency of printing the report.

		Return the optimal weight by gradient descent and
		the corresponding learning objectives.
	'''
	m, n = X.shape
	k = y.shape[1]
	nll_list = []
	# initialize the weight and its gradient
	W = np.zeros((n, k))
	W_grad = np.ones((n, k))
	print('==> Running gradient descent...')
	iter_num = 0
	t_start = time.time()
	# Running the gradient descent algorithm
	# Update W
	# Calculate learning objectives
	# YOUR CODE GOES BELOW
	while iter_num < max_iter and np.linalg.norm(W_grad) > eps:
		# calculate NLL
		nll = NLL(X, y, W, reg=reg)
		if np.isnan(nll):
			break
		nll_list.append(nll)
		# calculate gradients and update W
		W_grad = grad_softmax(X, y, W, reg=reg)
		W -= lr * W_grad

		if (iter_num + 1) % print_freq == 0:
			print('-- Iteration {} - \
				negative log likelihood {: 4.4f}'.format(iter_num + 1, nll))
		iter_num += 1
	# benchmark
	t_end = time.time()
	print('-- Time elapsed for running gradient descent: {t:2.2f} \
		seconds'.format(t=t_end - t_start))

	return W, nll_list

def accuracy_vs_lambda(X_train, y_train_OH, X_test, y_test, lambda_list):
	'''
		Generate accuracy for all given regularization parameters.
		Generate a plot of accuracy vs lambda.
		Return the lambda with optimal accuracy.
	'''
	# Find corresponding accuracy values for each parameter
	accu_list = []
	# YOUR CODE BELOW
	for reg in lambda_list:
		W, nll_list = grad_descent(X_train, y_train_OH, reg=reg, lr=2e-5, \
		print_freq=50)
		y_pred = predict(X_test, W)
		accuracy = get_accuracy(y_pred, y_test)
		accu_list.append(accuracy)
		print('-- Accuracy is {:2.4f} for lambda = {:2.2f}'.format(accuracy, reg))
	# Plot accuracy vs lambda
	print('==> Printing accuracy vs lambda...')
	plt.style.use('ggplot')
	plt.plot(lambda_list, accu_list)
	plt.title('Accuracy versus Lambda in Softmax Regression')
	plt.xlabel('Lambda')
	plt.ylabel('Accuracy')
	plt.savefig('hw4pr2b_lva.png', format = 'png')
	plt.close()
	print('==> Plotting completed.')
	# Find optimal lambda
	opt_lambda_index = np.argmax(accu_list)
	reg_opt = lambda_list[opt_lambda_index]
	return reg_opt


In [26]:
df_train = data.df_train
df_test = data.df_test
'''
    X is a matrix with dimension m x n
    y is a vector with dimension m x 1
'''
X_train = data.X_train
y_train = data.y_train
X_test = data.X_test
y_test = data.y_test
# stacking an array of ones
X_train = np.hstack((np.ones_like(y_train), X_train))
X_test = np.hstack((np.ones_like(y_test), X_test))
# one hot encoder
enc = OneHotEncoder()
y_train_OH = enc.fit_transform(y_train.copy()).astype(int).toarray()
y_test_OH = enc.fit_transform(y_test.copy()).astype(int).toarray()

In [27]:
lambda_list = [0.01, 0.1, 0.5, 1.0, 10.0, 50.0, 100.0, 200.0, 500.0, 1000.0]
reg_opt = accuracy_vs_lambda(X_train, y_train_OH, X_test, y_test, lambda_list)
print('-- Optimal regularization parameter is {:2.2f}'.format(reg_opt))

W_gd, nll_list_gd = grad_descent(X_train, y_train_OH, reg=reg_opt, max_iter=1500, lr=2e-5, \
    print_freq=100)

plt.style.use('ggplot')
# Plot the learning curve of NLL vs Iteration
# YOUR CODE BELOW
nll_gd_plot, = plt.plot(range(len(nll_list_gd)), nll_list_gd)
plt.setp(nll_gd_plot, color = 'red')
plt.title('Convergence Plot on Softmax Regression with $\lambda = {:2.2f}$'.format(reg_opt))
plt.xlabel('Iteration')
plt.ylabel('NLL')
plt.savefig('hw4pr2b_convergence.png', format = 'png')
plt.close()

==> Running gradient descent...
-- Iteration 50 - 				negative log likelihood  22498.2902
-- Iteration 100 - 				negative log likelihood  20132.7688
-- Iteration 150 - 				negative log likelihood  19074.1782
-- Iteration 200 - 				negative log likelihood  18420.1137
-- Iteration 250 - 				negative log likelihood  17962.7355
-- Iteration 300 - 				negative log likelihood  17618.8274
-- Iteration 350 - 				negative log likelihood  17347.3245
-- Iteration 400 - 				negative log likelihood  17125.3589
-- Iteration 450 - 				negative log likelihood  16939.0725
-- Iteration 500 - 				negative log likelihood  16779.5179
-- Time elapsed for running gradient descent: 76.66 		seconds
-- Accuracy is 0.9221 for lambda = 0.01
==> Running gradient descent...
-- Iteration 50 - 				negative log likelihood  22502.8732
-- Iteration 100 - 				negative log likelihood  20138.7176
-- Iteration 150 - 				negative log likelihood  19081.3517
-- Iteration 200 - 				negative log likelihood  18428.3600
-- Iterati